# 1. Clusterización K-Means

**Objetivo:** hacer una clusterización de sismos históricos globales.

El centro nacional de información de sismos (NEIC) determina la ubicación y magnitud de los sismos más importantes que han ocurrido en el mundo y disemina esta información inmediatament a agencias nacionales e internacionales. 

https://www.kaggle.com/usgs/earthquake-database

**Información de las características**
This dataset includes a record of the date, time, location, depth, magnitude, and source of every earthquake with a reported magnitude 5.5 or higher since 1965.

* 0 Date
* 1 Time
* 2 Location
* 3 Depth
* 4 Magnitude
* 5 Source of the earthquake


**Número de instancias:** 23412

# 2. Autenticación de drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# 3. Importando librerías

In [ ]:
import ____ as pd
import os 
import _____as plt
import _____ as sns
import _____ as np
import plotly.graph_objects as go
import plotly.express as ___
import plotly.io as ____
from sklearn.____ import KMeans
from sklearn.____ import StandardScaler

# 4. Lectura del archivo y visualización de los datos

In [ ]:
path = _________
name = 'NEIC_Earthquakes.csv'

In [ ]:
sis_path = os.____.join(____,_____)
sismosdf = pd._____(sis_path)
sismosdf.head()

Observemos que tenemos muchos campos con NaN, ¿Eliminamos las filas con NaN o las columnas?.
Visualizaremos que tan vacío está el dataframe con un mapa de calor

In [ ]:
fig, ax = plt.subplots(figsize=(25,8))
sns._____(_____._____(), ax=ax, cmap="magma")
plt.grid()
plt.show()

In [ ]:
_____.shape

# 5. Limpieza de los datos

Eliminar columnas con mayoría de campos indefinidos. **Quedemonos solo con Latitude	Longitude	Depth	Magnitude**

In [ ]:
sismosdf.drop([____], axis=1, inplace=True)

In [ ]:
print(sismosdf.shape)
_____.head()

Eliminación de instancias (filas) con valores idefinidos

In [ ]:
sismosdf._____(axis=0, inplace=True)

In [ ]:
sismosdf.shape

# 6. Exploración de los datos de sismos históricos

#### Revisando las columnas: Type, Magnitude



##### Columna de magnitud

In [ ]:
fig, ax = plt.subplots(figsize=(25,5))
sns.countplot(ax=ax, x=sismosdf[_____], color='darkorange')
plt.show()

##### Columna Type

In [ ]:
sismosdf[____].value_counts().plot(kind='bar', figsize=(10,5), grid=True, color='hotpink')

Eliminación de sismos de otro tipo diferente de 'Earthquake'

In [ ]:
sismosdf = sismosdf[sismosdf.____ == ____]
sismosdf.drop(['Type'], axis=1, inplace=True)

In [ ]:
sismosdf.head()

Descripción de los estadísticos de las 4 variables

In [ ]:
sismosdf.describe()

Mapa de Magnitud

In [ ]:
fig = go.Figure(data=go.Scattergeo(
        lon = sismosdf[____],
        lat = sismosdf[____],
        text = sismosdf['Depth'],
        marker = dict(
        color = sismosdf['Magnitude'],
        colorscale = 'Rainbow',
        reversescale = True,
        opacity = 0.7,
        size = 2,
        colorbar = dict(
            titleside = "right",
            outlinecolor = "rgba(68, 68, 68, 0)",
            ticks = "outside",
            showticksuffix = "last",
            dtick = 0.1
        )
    )
        ))
fig.update_traces(marker=dict(size=5),
                  selector=dict(mode='markers'))
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=25),
    title = 'Sismos históricos',
    )
fig.show()

# Normalización

In [ ]:
scaler = ______()
sismosdf.iloc[:,:] = _____._____(_____)
sismosdf.head()

# 7. Creación del modelo de clusterización

In [ ]:
g = ____.PairGrid(data=_____, height=2)
g.map_diag(plt.hist)
g.map_offdiag(plt.scatter)
g.add_legend()

Enfoquémonos en Depth vs Magnitude

In [ ]:
fig = px.scatter(data_frame=_____,
                    x=____,
                    y=____,
                    template='seaborn',
                    title='Depth vs Magnitude',
)
fig.update_layout(margin=dict(l=0, r=0, b=0, t=25))
fig.update_traces(marker=dict(size=8),
                  selector=dict(mode='markers'))
pio.show(fig)

**Np array con 2 variables**

In [ ]:
X = np.array(sismosdf[['Depth','Magnitude']])

#### Determinar el número de clusters adecuados

In [ ]:
Nc = range(1, 10)
SSD = []
for i in Nc:
  kmeans = KMeans(n_clusters=____) 
  kmeans = kmeans.fit(____)
  SSD.append(____.inertia_)


fig=go.Figure(layout=go.Layout(
        title=go.layout.Title(text="Gráfica del codo")
    ))
fig.update_xaxes(title_text='Número de clusters')
fig.update_yaxes(title_text='Suma de distancias al cuadrado')
fig.add_scatter(x=pd.Series(Nc), y=pd.Series(SSD), mode='lines')
fig.update_layout(width=500, height=400, margin=dict(l=0, r=0, b=0, t=25))
pio.show(fig)

#### Ajuste de clusterización y cálculo de centroides

In [ ]:
kmeans = KMeans(n_clusters=___)
kmeans = kmeans.____(___)
centroids = ____.cluster_centers_
print(centroids)

#### Cálculo de clusters

In [ ]:
labels = kmeans.predict(X)
print(pd.Series(labels).unique())

#### Creación de nuevo dataframe con etiquetas de clusters

In [ ]:
sismosdf['Clusters'] = labels
sismosdf.head()

#### Visualización de los clusters

In [ ]:
fig = px.scatter(data_frame=sismosdf,
                    x=    ____,
                    y=    ____,
                    color=____,
                    color_continuous_scale=["red", "green", "blue"],
                    template='seaborn',
                    title='Clusterización de sismos')
fig.update_layout(margin=dict(l=0, r=0, b=0, t=25))
fig.update_traces(marker=dict(size=6),
                  selector=dict(mode='markers'))
fig.add_scatter(x=pd.Series(centroids[:,0]), y=pd.Series(centroids[:,1]), 
                mode='markers', marker = dict(size = 20, color = 'magenta', symbol = 'star'), name="centroide")

pio.show(fig)